# Notebook for catboost model development and testing

scores, frequency 15min: 
training on observed data only 
- 173 med objective="MultiRSME", learning_rate=0.10931167365445349, colsample_bylevel=0.09963123954233088, max_depth=15, boosting_type="Plain", bootstrap_type="MVS"
- 171 med objective="MAE", learning_rate=0.10931167365445349, colsample_bylevel=0.09963123954233088, max_depth=15, boosting_type="Plain", bootstrap_type="MVS"
- 180 med vanlig catboost, trent på time series data. 

training on both observed and estimated 


## Getting data

trying 
- observed data as our training data
- estimated data as our validation data
- estimated testing as our test data 

## Training on observed and estimated data seperatly 

In [89]:
%store -r dm 

from sklearn.model_selection import train_test_split

dm = dm

X_A_train = dm.data_A_obs.iloc[:, 2:-1]
y_A_train = dm.data_A_obs.iloc[:,1]
X_B_train = dm.data_B_obs.iloc[:, 2:-1]
y_B_train = dm.data_B_obs.iloc[:,1]
X_C_train = dm.data_C_obs.iloc[:, 2:-1]
y_C_train = dm.data_C_obs.iloc[:,1]

X_A_val = dm.data_A_es.iloc[:, 2:-1]
y_A_val = dm.data_A_es.iloc[:,1]
X_B_val = dm.data_B_es.iloc[:, 2:-1]
y_B_val = dm.data_B_es.iloc[:,1]
X_C_val = dm.data_C_es.iloc[:, 2:-1]
y_C_val = dm.data_C_es.iloc[:,1]

print(X_C_train.shape, y_C_train.shape)

X_A_submission = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_submission = dm.X_test_estimated_b[dm.X_test_estimated_b.columns.intersection(X_B_train.columns)]
X_C_submission = dm.X_test_estimated_c[dm.X_test_estimated_c.columns.intersection(X_C_train.columns)]


(23141, 26) (23141,)


## Training on combined observed and estimated data

In [90]:
X_A_train = dm.data_A.iloc[:, 2:-1]
y_A_train = dm.data_A.iloc[:,1]
X_B_train = dm.data_B.iloc[:, 2:-1]
y_B_train = dm.data_B.iloc[:,1]
X_C_train = dm.data_C.iloc[:, 2:-1]
y_C_train = dm.data_C.iloc[:,1]

X_train_A, X_test_A, y_train_A, y_test_A = train_test_split(X_A_train, y_A_train, shuffle=True)
X_train_B, X_test_B, y_train_B, y_test_B = train_test_split(X_B_train, y_B_train, shuffle=True)
X_train_C, X_test_C, y_train_C, y_test_C = train_test_split(X_C_train, y_C_train, shuffle=True)

X_A_submission = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_submission = dm.X_test_estimated_b[dm.X_test_estimated_b.columns.intersection(X_B_train.columns)]
X_C_submission = dm.X_test_estimated_c[dm.X_test_estimated_c.columns.intersection(X_C_train.columns)]


## Using timeseries split to split our data 

In [91]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=10,
    gap=15,
    max_train_size=10000,
    test_size=1000,
)

all_splits_A = list(ts_cv.split(X_A_train, y_A_train))
all_splits_B = list(ts_cv.split(X_B_train, y_B_train))
all_splits_C = list(ts_cv.split(X_C_train, y_C_train))




In [1]:
from catboost import CatBoostRegressor

catModel_A = CatBoostRegressor()
catModel_B = CatBoostRegressor()
catModel_C = CatBoostRegressor()

## training on the time series split 

In [2]:

from sklearn.metrics import mean_absolute_error
import pandas as pd

for train, test in (all_splits_A): 

    catModel_A.fit(X_A_train.iloc[train], y_A_train.iloc[train], silent=True, use_best_model=True, eval_set=(X_A_train.iloc[test], y_A_train.iloc[test]))

    preds_A = pd.DataFrame(catModel_A.predict(X_A_train.iloc[test]))
    actual = y_A_train.iloc[test]

    print("CURRENT SCORING A ----->   ", mean_absolute_error(actual, preds_A))

for train, test in (all_splits_B): 

    catModel_B.fit(X_B_train.iloc[train], y_B_train.iloc[train], silent=True, use_best_model=True, eval_set=(X_B_train.iloc[test], y_B_train.iloc[test]))

    preds_B = pd.DataFrame(catModel_B.predict(X_B_train.iloc[test]))
    actual = y_B_train.iloc[test]

    print("CURRENT SCORING B ----->   ", mean_absolute_error(actual, preds_B))

for train, test in (all_splits_C): 

    catModel_C.fit(X_C_train.iloc[train], y_C_train.iloc[train], silent=True, use_best_model=True, eval_set=(X_C_train.iloc[test], y_C_train.iloc[test]))

    preds_C = pd.DataFrame(catModel_C.predict(X_C_train.iloc[test]))
    actual = y_C_train.iloc[test]

    print("CURRENT SCORING C ----->   ", mean_absolute_error(actual, preds_C))


NameError: name 'all_splits_A' is not defined

CURRENT SCORING A ----->    238.2081115564069
CURRENT SCORING A ----->    307.4649532526857
CURRENT SCORING A ----->    348.72257215076803
CURRENT SCORING A ----->    318.6929348355071
CURRENT SCORING A ----->    254.47124290495884
CURRENT SCORING A ----->    133.87777225370743
CURRENT SCORING A ----->    60.36754405596306
CURRENT SCORING A ----->    31.40668219331433
CURRENT SCORING A ----->    143.7804695768895
CURRENT SCORING A ----->    284.05657791089664
CURRENT SCORING B ----->    85.55463551332689
CURRENT SCORING B ----->    33.27661702464657
CURRENT SCORING B ----->    5.161436247523319
CURRENT SCORING B ----->    4.224951587201841
CURRENT SCORING B ----->    13.027637047945273
CURRENT SCORING B ----->    60.37406738312857
CURRENT SCORING B ----->    52.440665972971246
CURRENT SCORING B ----->    7.277092049872223
CURRENT SCORING B ----->    15.441143971740457
CURRENT SCORING B ----->    66.8439742097018
CURRENT SCORING C ----->    52.20386854042028
CURRENT SCORING C ----->    

In [94]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

for train, test in all_splits_A: 
    preds_A = pd.DataFrame(catModel_A.predict(X_A_train.iloc[test]))
    actual = y_A_train.iloc[test]

    print(mean_absolute_error(actual, preds_A))



184.4962396146782
256.44249046937244
288.5020896637525
266.4465593445813
201.70556159341174
127.5494374866649
35.809863582953604
21.81419548022472
92.16732959607698
260.16515976622156


In [110]:
from sklearn.metrics import mean_absolute_error
import pandas as pd


pred_A = pd.DataFrame(catModel_A.predict(X_test_A))
pred_B = pd.DataFrame(catModel_B.predict(X_test_B))
pred_C = pd.DataFrame(catModel_C.predict(X_test_C))

print(mean_absolute_error(y_test_A, pred_A))
print(mean_absolute_error(y_test_B, pred_B))
print(mean_absolute_error(y_test_C, pred_C))

preds = pd.concat([pred_A, pred_B, pred_C], ignore_index=True)
gt = pd.concat([y_test_A, y_test_B, y_test_C], ignore_index=True)

print(mean_absolute_error(gt, preds))



200.7840277308012
44.401694483579156
26.608341457261993
96.71783867686663


In [106]:
import pandas as pd

(pd.Series(catModel_A.feature_importances_, index=X_A_test.columns)
   .nlargest(20)
   .plot(kind='barh')) 

TypeError: Cannot use method 'nlargest' with dtype object

In [109]:
## Specify the model to be used
pred_A = catModel_A.predict(X_A_submission)
pred_B = catModel_B.predict(X_B_submission)
pred_C = catModel_C.predict(X_C_submission)
 
test = pd.read_csv('test.csv')

df_A = pd.DataFrame()

df_A["prediction"] = pred_A
df_A["location"] = "A"

df_B = pd.DataFrame()

df_B["prediction"] = pred_B
df_B["location"] = "B"

df_C = pd.DataFrame()

df_C["prediction"] = pred_C
df_C["location"] = "C"

df_mid = pd.concat([df_A, df_B], ignore_index=True)

df = pd.concat([df_mid, df_C], join="inner", ignore_index=True)



df = df.drop("location", axis=1)


# df["id"] = test["id"]

# df = df[["id", "prediction"]]

#df[df<0] = 0

# NAME THE FILE 
df.to_csv("sub33.csv")

df

,prediction
0,1.142758e+01
1,1.164954e+01
2,1.538397e+01
3,7.666517e+01
4,3.583734e+02
...,...
2155,3.432572e+01
2156,7.710396e+00
2157,2.305723e-01
2158,-1.147154e-07
